# Finetune RuGPTs in megatron without deepspeed
How to finetune RuGPTs models with megatron without deepspeed. Example for RuGPT3Small. Note for other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [ ]:
!pip3 install transformers==3.5.0

In [3]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Writing setup.sh


In [ ]:
!sh setup.sh

In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

## Download files

In [ ]:
!wget -O train.txt https://www.dropbox.com/s/oa3v9c7g9bp40xw/train.txt?dl=0
!wget -O valid.txt https://www.dropbox.com/s/mworl3ld6r3bg62/valid.txt?dl=0

## Prepare data for parallel
We use custom implementation of distributed dataset. For training and evaluating we should specify file `file.list` with list of paths to txt files. All files from `file.list` will be splitted between aviable GPUs. The logic of splitting is described by the following code:

```python
shard_size = len(files) // world_size
shard_start = rank * shard_size
shard_end = (rank + 1) * shard_size
files = files[shard_start:shard_end]
```

For more details please see full code of dataset: `src.dataset_rugpt3.RuGpt3TextDataset`.

In [9]:
!echo train.txt > train.list
!echo valid.txt > valid.list

## Train
Load model from Huggingface and finetune on essays.

This will take arount ten minutes.

In [ ]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/pretrain_gpt3.py \
  --train-data-path "train.list" \
  --test-data-path "valid.list" \
  --max-files-per-process 100 \
  --logging-dir="log" \
  --save model \
  --load-huggingface sberbank-ai/rugpt3small_based_on_gpt2 \
  --save-interval 1000 \
  --log-interval 100 \
  --eval-interval 1000 \
  --eval-iters 100 \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --batch-size 1 \
  --seq-length 512 \
  --max-position-embeddings 2048 \
  --train-iters 2000 \
  --resume-dataloader \
  --distributed-backend "nccl" \
  --lr 0.00015 \
  --lr-decay-style "cosine" \
  --lr-decay-iters 3200 \
  --clip-grad 0.5 \
  --warmup .004


At the end of training output should be something like this:

"-----------------------------------------------------------------------------------------

 validation loss at the end of training for test data | LM loss: 2.7927 | LM PPL: 16.325
 
-----------------------------------------------------------------------------------------"

## Generate

Load pretrained model from dir and generate.

In [29]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/generate_samples.py \
  --load model/ \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --batch-size 1 \
  --seq-length 512 \
  --max-position-embeddings 2048 \
  --distributed-backend "nccl" \
  --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
  --no-load-optim


2021-02-11 21:33:07.459111: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Generate Samples
using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling
> initializing model parallel with size 1
> initializing model parallel cuda seeds on global rank 0, model parallel rank 0, and data parallel rank 0 with model parallel seed: 3952 and data parallel seed: 1234
prepare tokenizer done, size 50264
building GPT3 model ...
 > number of parameters on model parallel rank 0: 125231616
Load checkpoint from model/
global rank 0 is loading checkpoint model/iter_0002000/mp_rank_00/model_optim_rng.pt
  successfully loaded model/iter_0002000/mp_rank_00/model_optim_rng.pt
Loaded

Context prompt (stop to exit) >>> <s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: 


Taken time 15.56


Context: <s>Тема: «Создает человека природа, но развивает и образует его общест

### Convert checkpoint to Huggingface format

In [37]:
!export PYTHONPATH=${PYTHONPATH}:${HOME}/ru-gpts

!python ru-gpts/convert2huggingface.py \
  --load model/ \
  --model-parallel-size 1 \
  --num-layers 12 \
  --hidden-size 768 \
  --num-attention-heads 12 \
  --max-position-embeddings 2048 \
  --tokenizer-path sberbank-ai/rugpt3small_based_on_gpt2 \
  --no-load-optim \
  --export-huggingface model_hf


2021-02-11 21:49:23.948606: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
using world size: 1 and model-parallel size: 1 
 > using dynamic loss scaling
> initializing model parallel with size 1
prepare tokenizer done, size 50264
building GPT3 model ...
 > number of parameters on model parallel rank 0: 125231616
Load checkpoint from model/
global rank 0 is loading checkpoint model/iter_0002000/mp_rank_00/model_optim_rng.pt
  successfully loaded model/iter_0002000/mp_rank_00/model_optim_rng.pt
Loaded
Export to huggingface model  model_hf with config {'vocab_size': 50264, 'n_positions': 2048, 'n_ctx': 2048, 'n_embd': 768, 'n_layer': 12, 'n_head': 12}
Saved huggingface model <class 'src.model.distributed.DistributedDataParallel'>
Exported in huggingface format to model_hf


In [38]:
!ls model_hf

config.json  pytorch_model.bin	      tokenizer_config.json
merges.txt   special_tokens_map.json  vocab.json


#### Test load

In [39]:
from transformers import GPT2LMHeadModel

In [40]:
model = GPT2LMHeadModel.from_pretrained("model_hf")